In [41]:
import numpy as np
import pandas as pd

In [42]:
bgvar_20 = pd.read_stata('avars_202001_EN_1.0p.dta')
hea_20 = pd.read_stata('ch20m_EN_1.0p.dta')
rel_20 = pd.read_stata('cr20m_EN_1.1p.dta')
soc_20 = pd.read_stata('cs20m_EN_1.1p.dta')
fam_20 = pd.read_stata('cf20m_EN_1.0p.dta')
wor_20 = pd.read_stata('cw20m_EN_1.0p.dta')
per_20 = pd.read_stata('cp20l_EN_1.0p.dta')
val_20 = pd.read_stata('cv20l_EN_1.0p.dta')
ass_20 = pd.read_stata('ca20g_EN_1.0p.dta')
inc_20 = pd.read_stata('ci20m_EN_1.0p.dta')
hou_20 = pd.read_stata('cd20m_EN_2.0p.dta')

In [43]:
#column combinators (replaces given columns with one containing their avg values)
def average_columns(df, *columns):
    new_column_name = columns[-1]
    
    try:
        selected_columns = list(columns[:-1])
        avg_col = df[selected_columns].astype(float).mean(axis=1)
        return pd.DataFrame({new_column_name: avg_col})
    except KeyError as e:
        print(f'KeyError: {e} not found in DataFrame columns.')
        return pd.DataFrame({new_column_name: np.nan})

#replaces given columns with one containing their sums (not working ;-())
def sum_columns(df, new_column, *columns):
    df[new_column] = df[columns].sum(axis=1)

Background variables:

1. geslacht to sex
2. leeftijd to age
3. lftdcat to age_cat
4. aantalhh to hhm_num
5. aantalki to hh_chi
6. burgstat to civ_sta
7. woning to dwe_typ
8. sted to urb_lvl (Urban character: Surrounding address density per km2 extremely urban 2,500 or more very 1,500 to 2,500 moderately 1,000 to 1,500 slightly 500 to 1,000 not less than 500) 
9. belbezig to pri_occ 
10. brutoink to mon_inc 
11. brutocat mon_inc_cat 12. brutohh_f to hh_inc 
13. oplzon to edu_lvl

In [44]:
bgvar_20[bgvar_20.select_dtypes(include='category').columns] = bgvar_20[bgvar_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
bg_var_names = {'nomem_encr':'nomem_encr', 'geslacht':'sex','leeftijd':'age', 'lftdcat':'age_cat', 'aantalhh':'hhm_num', 'aantalki':'hh_chi', 'burgstat':'civ_sta', 'woning':'dwe_typ', 'sted':'urb_lvl',  'belbezig':'pri_occ', 'brutoink':'mon_inc', 'brutocat':'mon_inc_cat','brutohh_f':'hh_inc', 'oplzon':'edu_lvl'}
bg_var = bgvar_20.loc[:, list(bg_var_names.keys())].rename(columns=bg_var_names)
bg_var.head()

,nomem_encr,sex,age,age_cat,hhm_num,hh_chi,civ_sta,dwe_typ,urb_lvl,pri_occ,mon_inc,mon_inc_cat,hh_inc,edu_lvl
0,800009.0,0,64.0,5,1,0,0,0,4,10,0,13,5495.900676,4
1,800015.0,0,57.0,5,3,2,0,0,0,0,942,9,6700.000000,5
2,800042.0,1,44.0,3,3,2,0,0,0,0,264,3,4269.000000,2
3,800054.0,1,73.0,6,0,0,2,1,3,8,0,13,1497.393186,1
4,800057.0,0,45.0,4,4,3,4,0,0,0,1147,12,16500.000000,5


Concepts in Health:
1. **Subjective health** (ch20m004 to sub_hea)
2. **Relative health** (ch20m005 to rel_hea)
3. Life expectancy
4. **Mental health** (all neg as men_hea_neg and and all pos as men_hea_pos)
5. Height, weight
6. **Chronical problems** (ch20m018 to disability)
7. Health problems
8. Impediment in labour
9. **Substance use** (ch20m126 to smoker, ch20m133 to alcohol, all substances to subs)
10. Medication
11. Physical activity - only until 2018
12. Health improving behaviour - - only until 2018
13. Use of healthcare
14. Sight and hearing
15. Health insurance

In [45]:
#select categorical columns and convert to numeric
hea_20[hea_20.select_dtypes(include='category').columns] = hea_20[hea_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

#average for mental health and drugs
hea_toavg = [('ch20m011','ch20m014','ch20m012', 'men_hea_neg'),('ch20m013', 'ch20m015', 'men_hea_pos'), ('ch20m159', 'ch20m160', 'ch20m161', 'ch20m161', 'ch20m162', 'ch20m163', 'ch20m270', 'subs')]
hea_20 = pd.concat([hea_20] + [average_columns(hea_20, *column_set) for column_set in hea_toavg], axis=1)

#create a new df hea_20_con with selected columnns and new columns containing avgs (representing concepts)
hea_con = {'nomem_encr':'nomem_encr', 'ch20m004': 'sub_hea', 'ch20m005': 'rel_hea', 'ch20m018': 'disability', 'ch20m020':'mob_issues', 'ch20m126':'smoker', 'ch20m133':'alcohol', 'men_hea_neg':'men_hea_neg', 'men_hea_pos':'men_hea_pos', 'subs':'subs'}
hea_20_con = hea_20.loc[:, list(hea_con.keys())].rename(columns=hea_con)

#ready
hea_20_con.head()

,nomem_encr,sub_hea,rel_hea,disability,mob_issues,smoker,alcohol,men_hea_neg,men_hea_pos,subs
0,800009.0,2,2,0,0,1,0,0.000000,3.5,0.000000
1,800015.0,2,2,0,0,-1,2,0.000000,3.5,0.000000
2,800057.0,3,1,1,0,1,1,0.666667,3.5,0.000000
3,800058.0,2,3,1,3,-1,3,2.333333,3.0,0.142857
4,800100.0,4,4,1,0,-1,7,0.000000,5.0,0.000000


Concepts in Religion and Ethnicity:

1. **Religious upbringing** (cr20m135 to rel_upb)
2. **Religious affiliation** (cr20m144 to rel_aff)
3. **Religiosity** (cr20m162 to relig)
4. Religious orthodoxy
5. Nationality 
6. **Origin** (cr20m079 – cr20m087 to lang_ori, cr20m079 indicating Dutch or not)
7. **Ethnic identification** (cr20m164 to eth_id)
8. Language proficiency and use

In [46]:
rel_con = {'nomem_encr':'nomem_encr','cr20m135':'rel_upb', 'cr20m144':'rel_aff', 'cr20m162':'relig', 'cr20m079':'lang_ori', 'cr20m164':'eth_id'}
rel_20[rel_20.select_dtypes(include='category').columns] = rel_20[rel_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
rel_20_con = rel_20.loc[:, list(rel_con.keys())].rename(columns=rel_con)
rel_20_con.head()

,nomem_encr,rel_upb,rel_aff,relig,lang_ori,eth_id
0,800009.0,3,2,2,1,1
1,800015.0,1,-1,4,0,3
2,800057.0,2,-1,4,1,0
3,800058.0,-1,-1,3,1,1
4,800085.0,2,-1,3,0,1


Concepts in Social Intergation and Leisure:

1. Loneliness
2. Social contacts 
3. **Satisfaction leisure** (cs20m001 to lei_sat)
4. Social engagement
5. Volunteer aid
6. Time expenditure voluntary work
7. **Cultural participation** (sum cs20m494 - cs20m496 to cul_par)
8. **Holidays** (cs20m103 to ab_hol)
9. **Sport** (cs20m105 to spo_h)
10. Media usage

In [47]:
soc_20[soc_20.select_dtypes(include='category').columns] = soc_20[soc_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
cultosum = ['cs20m494', 'cs20m495', 'cs20m517', 'cs20m093', 'cs20m094', 'cs20m568', 'cs20m516', 'cs20m496']
soc_20['cul_sum'] = soc_20[cultosum].sum(axis=1)
soc_con = {'nomem_encr':'nomem_encr','cs20m001':'lei_sat', 'cul_sum':'cul_par', 'cs20m103':'ab_hol', 'cs20m105':'spo_h'}
soc_20_con = soc_20.loc[:, list(soc_con.keys())].rename(columns=soc_con)
print(soc_20_con)

      nomem_encr  lei_sat  cul_par  ab_hol  spo_h
0       800009.0        9        4       0    NaN
1       800015.0        9        0       1    NaN
2       800058.0        9        3       1    2.0
3       800100.0       11        0       0   10.0
4       800119.0        9        0       1    NaN
...          ...      ...      ...     ...    ...
5965    899794.0       11        8       0    2.0
5966    899908.0        5        0       0    8.0
5967    899923.0        8        2       2    6.0
5968    899928.0        8        5       0    3.0
5969    899946.0        6        3       0    NaN

[5970 rows x 5 columns]


/var/folders/rc/7v4gm02n4dl8nxs5j8gsxyz80000gn/T/ipykernel_31167/4126897073.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  soc_20['cul_sum'] = soc_20[cultosum].sum(axis=1)


Concepts in Family and Household:

1. **Parental relations** (cf20m145+cf20m146 to par_int)
2. **Partnership** (cf20m025 to liv_tog)
3. Marriage
4. Marital history
5. **Children** (cf20m455 to chi_num, cf20m456 to chi1_bir)
6. **Partnership and childbirth intentions** (cf20m128 to chi_fut)
7. Infertility
8. Parental support
9. Parent support
10. Quality of relationship
11. Division of domestic tasks
12. Perceived burden domestic tasks
13. Perceived burden child care
14. Perceived fairness division tasks
15. Child education: Supervision
16. Childcare

In [48]:
fam_20[fam_20.select_dtypes(include='category').columns] = fam_20[fam_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
famtosum = ['cf20m143', 'cf20m144']
fam_20['fam_sum'] = fam_20[famtosum].sum(axis=1)
fam_con = {'nomem_encr':'nomem_encr', 'fam_sum':'par_int', 'cf20m025':'liv_tog', 'cf20m455':'chi_num', 'cf20m456':'chi1_bir', 'cf20m128':'chi_fut'}
fam_20_con = fam_20.loc[:, list(fam_con.keys())].rename(columns=fam_con)
print(fam_20_con)

      nomem_encr  par_int  liv_tog  chi_num  chi1_bir  chi_fut
0       800009.0       -2        0      3.0    1982.0       -1
1       800015.0       -2        0      2.0    1994.0       -1
2       800057.0        4        0      3.0    2007.0        1
3       800058.0        2       -1      NaN       NaN        0
4       800085.0       -2       -1      NaN       NaN        0
...          ...      ...      ...      ...       ...      ...
6071    899794.0       -2        0      2.0    1993.0       -1
6072    899908.0        4       -1      NaN       NaN        0
6073    899923.0        4        0      2.0    1993.0       -1
6074    899928.0       -2       -1      1.0    1970.0       -1
6075    899946.0        4       -1      NaN       NaN        1

[6076 rows x 6 columns]


/var/folders/rc/7v4gm02n4dl8nxs5j8gsxyz80000gn/T/ipykernel_31167/4237264192.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fam_20['fam_sum'] = fam_20[famtosum].sum(axis=1)


Concepts in Work and Schooling:

1. Employment / activity status
2. Hours of paid work
3. **Characteristics current or last job** (cw20m136 to wor_min, cw20m140 to wor_nig, cw20m142 to wor_hom, cw20m402 to wor_sect)
4. Average pay job
5. Second jobs
6. Work aspirations
7. **Satisfaction** (cw20m128 to inc_sat)
8. Employment conditions
9. Commuting
10. Pensions
11. Non-working, non-pension
12. Job search
13. Satisfaction education
14. Highest qualification
15. Highest education
16. Qualifications for job
17. Education and training in last year

cw22o439 to chi_<8

In [49]:
wor_20[wor_20.select_dtypes(include='category').columns] = wor_20[wor_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
wor_con = {'nomem_encr':'nomem_encr', 'cw20m136':'wor_min', 'cw20m140':'wor_nig', 'cw20m142':'wor_hom', 'cw20m402':'wor_sect', 'cw20m128':'inc_sat', 'cw20m439':'chi_<8'}
wor_20_con = wor_20.loc[:, list(wor_con.keys())].rename(columns=wor_con)
print(wor_20_con)

      nomem_encr  wor_min  wor_nig  wor_hom  wor_sect  inc_sat  chi_<8
0       800009.0      NaN       -1       -1        -1       -1      -1
1       800015.0      5.0        1        0        10        7       1
2       800057.0     15.0        1        0        -1        7      -1
3       800058.0      5.0        0        0         6        5      -1
4       800100.0      5.0       -1        0        12        7       0
...          ...      ...      ...      ...       ...      ...     ...
5641    899928.0      NaN       -1       -1        -1       -1      -1
5642    899934.0      NaN       -1       -1        14       -1      -1
5643    899946.0      NaN       -1       -1        14       -1      -1
5644    899954.0      NaN       -1       -1        -1       -1      -1
5645    899960.0     55.0        0        0         9        6      -1

[5646 rows x 7 columns]


Concepts in Personality:
1. Happiness
2. **Life satisfaction** (cp20m014-016 to life_satf, cp20m070-079 as self_est_pos or self_est_neg)
3. Big five personality
4. Survey attitude
5. Self esteem
6. Mood
7. Need to evaluate
8. **Value orientation** (cp20m100 to res, cp20m110 to ind, cp20m118 to fam_sec, cp20m119 to free, cp20m129 to comf_life, cp20m133 to soc_rec)
9. Inclusion of Others in the Self
10. Social desirability
11. Affects
12. Need for Cognition
13. **Social trust** (cp20m019 to trust )

In [50]:
per_20[per_20.select_dtypes(include='category').columns] = per_20[per_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

#averages for life satisfaction and self-esteem
per_toavg = [('cp20l014','cp20l015','cp20l016', 'cp20l017', 'cp20l018','life_satf'),('cp20l070', 'cp20l071', 'cp20l073', 'cp20l075', 'cp20l076', 'self_est_pos'), ('cp20l072', 'cp20l074', 'cp20l078', 'cp20l079', 'self_est_neg')]
per_20 = pd.concat([per_20] + [average_columns(per_20, *column_set) for column_set in per_toavg], axis=1)

#create a new df per_20_con with selected columnns and new columns containing avgs (representing concepts)
per_con = {'nomem_encr':'nomem_encr','cp20l100': 'resp', 'cp20l110': 'ind','cp20l118': 'fam_sec','cp20l119': 'free', 'cp20l133': 'soc_rec', 'cp20l129': 'comf_life', 'cp20l019': 'trust', 'life_satf':'life_satf', 'self_est_pos':'self_est_pos', 'self_est_neg':'self_est_neg' }
per_20_con = per_20.loc[:, list(per_con.keys())].rename(columns=per_con)

#ready
per_20_con.head()

,nomem_encr,resp,ind,fam_sec,free,soc_rec,comf_life,trust,life_satf,self_est_pos,self_est_neg
0,800009.0,-1,-1,-1,-1,-1,-1,7,3.2,-1.0,-1.00
1,800015.0,6,6,6,6,0,5,6,4.2,5.2,0.00
2,800057.0,5,6,6,6,4,5,8,5.4,5.4,0.00
3,800058.0,5,3,6,6,3,5,9,4.8,4.4,2.25
4,800085.0,6,4,-1,-1,-1,-1,7,3.6,6.0,0.00


Concepts in Values:
1. Beliefs about maternal employment
2. **Gender role attitudes** (cv20l113 to gend_roles)
3. Political position ethnic minorities *page not working*
4. Right wing attitudes *page not working*
5. **Attitude towards foreigners** (cv20l120 to nationalist)
6. Marriage attitudes
7. Family solidarity
8. Norm of having children
9. Work ethic
10. Conservatism
11. Economic conservatism

In [51]:
val_20[val_20.select_dtypes(include='category').columns] = val_20[val_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

#create a new df val_20_con with selected columnns and new columns containing avgs (representing concepts)
val_con = {'nomem_encr':'nomem_encr','cv20l113': 'gend_roles', 'cv20l120': 'nationalist'}
val_20_con = val_20.loc[:, list(val_con.keys())].rename(columns=val_con)

#ready
val_20_con.head()

,nomem_encr,gend_roles,nationalist
0,800009.0,0,2
1,800015.0,0,0
2,800054.0,-1,-1
3,800057.0,0,3
4,800085.0,-1,-1


Concepts in Economic Situtation: Assets:
1. **Assets** (ca20g008 to ass_own, ca20g023 to ass_val, ca20g012 to tot_bal)

In [52]:
ass_20['ca20g023'] = pd.to_numeric(ass_20['ca20g023'], errors='coerce')
ass_20[ass_20.select_dtypes(include='category').columns] = ass_20[ass_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)
ass_con = {'nomem_encr':'nomem_encr', 'ca20g008':'ass_own','ca20g023':'ass_val', 'ca20g012':'tot_bal'}
ass_20_con = ass_20.loc[:, list(ass_con.keys())].rename(columns=ass_con)

ass_20_con.head()

,nomem_encr,ass_own,ass_val,tot_bal
0,800009.0,1,1500.0,6.900000e+03
1,800015.0,0,NaN,1.000000e+04
2,800057.0,1,-9.0,-1.000000e+10
3,800058.0,0,NaN,2.000000e+03
4,800100.0,1,-8.0,-1.000000e+10


Concepts in Income: 
1. Subjective standard of living
2. Satisfaction income
3. Employment income / Earnings cj008
4. Self-employment income
5. Non-employment income: pension
6. Non-employment income: benefits
7. Non-employment income: other
8. Credit, debt, payments
9. **Total income** (ci20m226 to income)
10. **Financial/material well-being** (ci20m252 to fin_sit)
11. Financial expectations
12. Financial management
13. Material deprivation


In [53]:
inc_20[inc_20.select_dtypes(include='category').columns] = inc_20[inc_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

#create a new df per_22_con with selected columnns and new columns containing avgs (representing concepts)
inc_con = {'nomem_encr':'nomem_encr','ci20m226': 'income', 'ci20m252': 'fin_sit' }
inc_20_con = inc_20.loc[:, list(inc_con.keys())].rename(columns=inc_con)

#ready
inc_20_con.head()

,nomem_encr,income,fin_sit
0,800015.0,-1,4
1,800058.0,0,0
2,800100.0,-1,4
3,800119.0,0,3
4,800127.0,8,1


Concepts for Housing:
1. Satisfaction housing
2. Housing characteristics
3. Housing expenditure 
4. **Housing wealth** (cd20m025 to pur_price)

In [54]:
hou_20[hou_20.select_dtypes(include='category').columns] = hou_20[hou_20.select_dtypes(include='category').columns].apply(lambda col: col.astype('category').cat.codes)

hou_con = {'nomem_encr':'nomem_encr','cd20m025': 'pur_price'}
hou_20_con = hou_20.loc[:, list(hou_con.keys())].rename(columns=hou_con)
hou_20_con.head()

,nomem_encr,pur_price
0,800015.0,94463.0
1,800058.0,NaN
2,800127.0,140000.0
3,800161.0,340000.0
4,800170.0,75000.0


In [55]:
#merge
all_20 = [fam_20_con, hea_20_con, soc_20_con, val_20_con, wor_20_con, inc_20_con, ass_20_con, hou_20_con] #bg_var, per_20_con

merged_20 = all_20[0]

for all in all_20[1:]:
    merged_20 = pd.merge(merged_20, all, on='nomem_encr', how='inner')

In [56]:
merged_20.head()

,nomem_encr,par_int,liv_tog,chi_num,chi1_bir,chi_fut,sub_hea,rel_hea,disability,mob_issues,...,wor_hom,wor_sect,inc_sat,chi_<8,income,fin_sit,ass_own,ass_val,tot_bal,pur_price
0,800015.0,-2,0,2.0,1994.0,-1,2,2,0,0,...,0,10,7,1,-1,4,0,NaN,1.000000e+04,94463.0
1,800170.0,-2,-1,3.0,1991.0,-1,2,2,1,2,...,0,12,2,1,10,4,0,NaN,1.200000e+04,75000.0
2,800186.0,-2,-1,3.0,1964.0,-1,2,3,1,1,...,-1,-1,-1,-1,7,4,0,NaN,-1.000000e+10,29000.0
3,800201.0,4,0,NaN,NaN,1,2,2,1,1,...,0,12,8,-1,-1,4,1,5000.0,4.500000e+04,NaN
4,800204.0,-2,0,2.0,1974.0,-1,2,3,0,0,...,-1,-1,-1,-1,7,1,1,15000.0,4.000000e+03,108000.0


In [58]:
from sklearn.impute import SimpleImputer

#replace empty values with NaN
merged_20.replace(' ', pd.NA, inplace=True)

#set nan to 0 for specific columns
to_0 = [ 'pur_price', 'chi1_bir'] #'rent'
merged_20[to_0] = merged_20[to_0].fillna(0)

#use SimpleImputer for numeric variables - tot_bal shouldnt probably be here (not accurate with mean)
numeric_vars = ['spo_h', 'wor_min', 'chi_num', 'tot_bal'] #'hh_inc', 'urb_lvl'
numeric_imputer = SimpleImputer(strategy='mean')
merged_20[numeric_vars] = numeric_imputer.fit_transform(merged_20[numeric_vars])

#drop na in urb_lvl
#merged_20.dropna(subset=['urb_lvl'], inplace=True)

KeyError: 'urb_lvl'